In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [5]:
from conlleval import evaluate

In [6]:
from ner_utils import *

Using TensorFlow backend.
/home/nlp/danb/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nlp/danb/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nlp/danb/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nlp/danb/anaconda3/e

## Get token data

In [7]:
import sys
sys.path.append('/home/nlp/danb')
import bclm

In [44]:
dev_lat = bclm.read_lattices(bclm.LATTICES_PATHS['dev'])
dev_lat.head(20)

,ID1,ID2,form,lemma,upostag,xpostag,feats,token_id,sent_id
0,0,1,עשרות,עשר,CDT,CDT,gen=F|num=P,1,1
1,0,1,עשרות,עשר,CD,CD,gen=F|num=P,1,1
2,1,2,אנשים,הנשים,VB,VB,gen=F|gen=M|num=S|per=1|tense=FUTURE,2,1
3,1,2,אנשים,איש,NN,NN,gen=M|num=P,2,1
4,2,3,מגיעים,הגיע,VB,VB,gen=M|num=P|per=A|tense=BEINONI,3,1
5,2,3,מגיעים,הגיע,BN,BN,gen=M|num=P|per=A,3,1
6,3,4,מ,מ,PREPOSITION,PREPOSITION,_,4,1
7,3,5,מתאילנד,מתאילנד,NNP,NNP,gen=M|num=S,4,1
8,3,5,מתאילנד,מתאילנד,NN,NN,gen=M|num=P|num=S,4,1
9,3,5,מתאילנד,מתאילנד,NN,NN,gen=M|num=S,4,1


In [47]:
dropped = [5438, 5444, 5445, 5446, 5448, 5449, 5450, 5451, 5453, 5459]
spdf = bclm.read_dataframe('spmrl')
spdf = spdf[(~spdf.sent_id.isin(dropped))]
tokens_ner_with_upos = bclm.get_token_df(spdf, fields = ['biose_layer0', 'upostag'])
tokens_ner_with_upos.head()

,sent_id,token_id,token_str,biose_layer0,upostag,set
0,1,1,עשרות,O,CDT,dev
1,1,2,אנשים,O,NN,dev
2,1,3,מגיעים,O,BN,dev
3,1,4,מתאילנד,O^S-GPE,PREPOSITION^NNP,dev
4,1,5,לישראל,O^S-GPE,PREPOSITION^NNP,dev


In [48]:
tok_gold_dev_sents =   tokens_ner_with_upos.groupby('sent_id')[['token_str', 'biose_layer0']].apply(lambda x: x.values.tolist())

In [49]:
def get_pred_sents(preds, gold_sents):
    bios = preds[0][3]
    pred_sents_l = [[i, list(zip(toks, bs))] 
                    for i, (toks, bs) 
                    in zip(gold_sents.index, (zip(preds[0][2], bios))) 
                    ]
    pred_sents = pd.Series(index=[idx for idx, s in pred_sents_l], data=[s for idx, s in pred_sents_l])
    return pred_sents

In [51]:
import pickle

In [52]:
eps = get_pred_sents(pickle.load(open('results/new_outputs_no_biose_fix/treebank_tokens_no_biose_fix-0-extra_preds-1.pkl', 'rb')), tok_gold_dev_sents)

In [53]:
ps = []
for i, sent in eps.iteritems():
    for j, (tok, bio) in enumerate(sent):
        ps.append([i, j+1, tok, bio, len(bio.split('^'))])
        
ps = pd.DataFrame(ps, columns=['sent_id', 'token_id', 'token_str', 'biose', 'biose_count'])
ps.head()

,sent_id,token_id,token_str,biose,biose_count
0,1,1,עשרות,O,1
1,1,2,אנשים,O,1
2,1,3,מגיעים,O,1
3,1,4,מתאילנד,O^S-GPE,2
4,1,5,לישראל,O^S-GPE,2


In [54]:
import networkx as nx

In [205]:
def get_valid_edges(non_o_only=True, keep_all_if_no_valid=False):
    valid_edges = []
    for (i, df), (_, biose, biose_count) in zip(dev_lat.groupby(['sent_id', 'token_id']), ps[['biose', 'biose_count']].itertuples()):
        el = df[['ID1', 'ID2']].rename(columns={'ID1': 'source', 'ID2': 'target'})
        #min_node = [n for n,v in G.nodes(data=True) if v['since'] == 'December 2008'][0]

        g = nx.from_pandas_edgelist(el, create_using=nx.DiGraph)
        min_node = el.source.min()
        max_node = el.target.max()
        #print(min_node,max_node)
        #print(biose_count)
        if non_o_only and not '-' in biose:
            vp = list(nx.all_simple_paths(g, min_node, max_node))
        else:
            vp = [path for path in nx.all_simple_paths(g, min_node, max_node, cutoff=biose_count+1) if len(path)==biose_count+1]
        if keep_all_if_no_valid and len(vp)==0:
             vp = nx.all_simple_paths(g, min_node, max_node)
        for path in vp:
            for source, target in zip(path[:-1], path[1:]):
                valid_edges.append((i[0], i[1], source, target))
                
    return valid_edges

In [ ]:
valid_edges_non_o = get_valid_edges()
valid_edges_all = get_valid_edges(non_o_only=False)

print(valid_edges_non_o[:30])

In [141]:
len(valid_edges_non_o), len(valid_edges_all)

(23910, 13238)

In [206]:
valid_edges_non_o_keep = get_valid_edges(keep_all_if_no_valid=True)
valid_edges_all_keep = get_valid_edges(non_o_only=False, keep_all_if_no_valid=True)

In [207]:
len(valid_edges_non_o_keep), len(valid_edges_all_keep)

(23932, 13479)

In [142]:
cols = ['sent_id', 'token_id', 'ID1', 'ID2']
dev_lat.groupby(cols).size().shape

(22742,)

In [143]:
pd.DataFrame(valid_edges_non_o).drop_duplicates().shape

(21984, 4)

In [144]:
#valid_edges = pd.DataFrame(valid_edges, columns=cols)

In [145]:
pruned_lat_non_o = dev_lat[dev_lat[cols].apply(lambda x: tuple(x) in valid_edges_non_o, axis=1)]

In [147]:
pruned_lat_all_ = dev_lat[dev_lat[cols].apply(lambda x: tuple(x) in valid_edges_all, axis=1)]

In [208]:
pruned_lat_non_o_keep = dev_lat[dev_lat[cols].apply(lambda x: tuple(x) in valid_edges_non_o_keep, axis=1)]

In [209]:
pruned_lat_all_keep = dev_lat[dev_lat[cols].apply(lambda x: tuple(x) in valid_edges_all_keep, axis=1)]

In [210]:
len(dev_lat), len(pruned_lat_non_o), len(pruned_lat_all), len(pruned_lat_non_o_keep), len(pruned_lat_all_keep)

(59758, 56941, 28451, 56979, 28951)

In [160]:
def to_lattices(df, path, cols = ['ID1', 'ID2', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'token_id']):
    with open(path, 'w', encoding='utf8') as of:
        for _, sent in df.groupby('sent_id'):
            for _, row in sent[cols].iterrows():
                of.write('\t'.join(row.astype(str).tolist())+'\n')
            of.write('\n')
            
    

In [160]:
to_lattices(pruned_lat_non_o, 'data/pruned/dev_pruned_non_o.lattices')

In [161]:
to_lattices(pruned_lat_all, 'data/pruned/dev_pruned_all.lattices')

In [211]:
to_lattices(pruned_lat_non_o_keep, 'data/pruned/dev_pruned_non_o_keep.lattices')

In [212]:
to_lattices(pruned_lat_all_keep, 'data/pruned/dev_pruned_all_keep.lattices')

## Run YAP

In [165]:
yap_path = '/home/nlp/danb/yapproj/src/yap/yap'

In [166]:
!export GOPATH=/home/nlp/danb/yapproj

In [167]:
!{yap_path}

/home/nlp/danb/yapproj/src/yap/yap - invoke yap as a standalone app or as an api server

Commands:

    api         start api server
    dep         runs dependency training/parsing
    hebma       run lexicon-based morphological analyzer on raw input
    joint       runs joint morpho-syntactic training and parsing
    ma          run data-driven morphological analyzer on raw input
    md          runs standalone morphological disambiguation training and parsing

Use "/home/nlp/danb/yapproj/src/yap/yap help <command>" for more information about a command.



In [168]:
import os
os.mkdir('data/pruned')

In [169]:
!{yap_path} joint -in data/pruned/dev_pruned_non_o.lattices -os data/pruned/dev_pruned_non_o.seg -om data/pruned/dev_pruned_non_o.map -oc data/pruned/dev_pruned_non_o.conll

2019/11/07 11:13:24.485763 GOMAXPROCS:	40
2019/11/07 11:13:24.486425 
2019/11/07 11:13:24.488078 *** CONFIGURATION ***
2019/11/07 11:13:24.488106 Beam:             	Standard Beam [Not Aligned & Not Averaged]
2019/11/07 11:13:24.488171 Transition System:	Joint Morpho-Syntactic [MD:Morpheme-Based Morphological Disambiguator, ArcSys:Arc Zeager (zpar acl '11) [a.k.a. ArcZEager]] - Strategy: ArcGreedy
2019/11/07 11:13:24.488209 Transition Oracle:	Joint Morpho-Syntactic - Strategy: ArcGreedy
2019/11/07 11:13:24.488229 Iterations:		1
2019/11/07 11:13:24.488252 Beam Size:		64
2019/11/07 11:13:24.488274 Beam Concurrent:	true
2019/11/07 11:13:24.488292 Parameter Func:	Funcs_Main_POS_Both_Prop
2019/11/07 11:13:24.488322 Use Lemmas:		false
2019/11/07 11:13:24.488345 Use POP:		true
2019/11/07 11:13:24.488369 Infuse Gold Dev:	false
2019/11/07 11:13:24.488387 Limit (thousands):	0
2019/11/07 11:13:24.488405 Use CoNLL-U:		false
2019/11/07 11:13:24.488441 
2019/11/07 11:13:24.488465 Features File:	joint

In [170]:
!{yap_path} joint -in data/pruned/dev_pruned_all.lattices -os data/pruned/dev_pruned_all.seg -om data/pruned/dev_pruned_all.map -oc data/pruned/dev_pruned_all.conll

2019/11/07 11:15:23.704949 GOMAXPROCS:	40
2019/11/07 11:15:23.705136 
2019/11/07 11:15:23.705795 *** CONFIGURATION ***
2019/11/07 11:15:23.705820 Beam:             	Standard Beam [Not Aligned & Not Averaged]
2019/11/07 11:15:23.705880 Transition System:	Joint Morpho-Syntactic [MD:Morpheme-Based Morphological Disambiguator, ArcSys:Arc Zeager (zpar acl '11) [a.k.a. ArcZEager]] - Strategy: ArcGreedy
2019/11/07 11:15:23.705901 Transition Oracle:	Joint Morpho-Syntactic - Strategy: ArcGreedy
2019/11/07 11:15:23.705931 Iterations:		1
2019/11/07 11:15:23.705951 Beam Size:		64
2019/11/07 11:15:23.705970 Beam Concurrent:	true
2019/11/07 11:15:23.705986 Parameter Func:	Funcs_Main_POS_Both_Prop
2019/11/07 11:15:23.706002 Use Lemmas:		false
2019/11/07 11:15:23.706020 Use POP:		true
2019/11/07 11:15:23.706039 Infuse Gold Dev:	false
2019/11/07 11:15:23.706055 Limit (thousands):	0
2019/11/07 11:15:23.706070 Use CoNLL-U:		false
2019/11/07 11:15:23.706088 
2019/11/07 11:15:23.706098 Features File:	joint

In [213]:
!{yap_path} joint -in data/pruned/dev_pruned_non_o_keep.lattices -os data/pruned/dev_pruned_non_o_keep.seg -om data/pruned/dev_pruned_non_o_keep.map -oc data/pruned/dev_pruned_non_o_keep.conll

2019/11/18 10:54:12.546276 GOMAXPROCS:	40
2019/11/18 10:54:12.546623 
2019/11/18 10:54:12.548333 *** CONFIGURATION ***
2019/11/18 10:54:12.548363 Beam:             	Standard Beam [Not Aligned & Not Averaged]
2019/11/18 10:54:12.548389 Transition System:	Joint Morpho-Syntactic [MD:Morpheme-Based Morphological Disambiguator, ArcSys:Arc Zeager (zpar acl '11) [a.k.a. ArcZEager]] - Strategy: ArcGreedy
2019/11/18 10:54:12.548417 Transition Oracle:	Joint Morpho-Syntactic - Strategy: ArcGreedy
2019/11/18 10:54:12.548436 Iterations:		1
2019/11/18 10:54:12.548456 Beam Size:		64
2019/11/18 10:54:12.548476 Beam Concurrent:	true
2019/11/18 10:54:12.548492 Parameter Func:	Funcs_Main_POS_Both_Prop
2019/11/18 10:54:12.548510 Use Lemmas:		false
2019/11/18 10:54:12.548530 Use POP:		true
2019/11/18 10:54:12.548550 Infuse Gold Dev:	false
2019/11/18 10:54:12.548567 Limit (thousands):	0
2019/11/18 10:54:12.548584 Use CoNLL-U:		false
2019/11/18 10:54:12.548605 
2019/11/18 10:54:12.548637 Features File:	joint

In [214]:
!{yap_path} joint -in data/pruned/dev_pruned_all_keep.lattices -os data/pruned/dev_pruned_all_keep.seg -om data/pruned/dev_pruned_all_keep.map -oc data/pruned/dev_pruned_all_keep.conll

2019/11/18 10:55:33.493698 GOMAXPROCS:	40
2019/11/18 10:55:33.493872 
2019/11/18 10:55:33.494511 *** CONFIGURATION ***
2019/11/18 10:55:33.494540 Beam:             	Standard Beam [Not Aligned & Not Averaged]
2019/11/18 10:55:33.494587 Transition System:	Joint Morpho-Syntactic [MD:Morpheme-Based Morphological Disambiguator, ArcSys:Arc Zeager (zpar acl '11) [a.k.a. ArcZEager]] - Strategy: ArcGreedy
2019/11/18 10:55:33.494609 Transition Oracle:	Joint Morpho-Syntactic - Strategy: ArcGreedy
2019/11/18 10:55:33.494628 Iterations:		1
2019/11/18 10:55:33.494650 Beam Size:		64
2019/11/18 10:55:33.494671 Beam Concurrent:	true
2019/11/18 10:55:33.494689 Parameter Func:	Funcs_Main_POS_Both_Prop
2019/11/18 10:55:33.494707 Use Lemmas:		false
2019/11/18 10:55:33.494729 Use POP:		true
2019/11/18 10:55:33.494751 Infuse Gold Dev:	false
2019/11/18 10:55:33.494777 Limit (thousands):	0
2019/11/18 10:55:33.494803 Use CoNLL-U:		false
2019/11/18 10:55:33.494825 
2019/11/18 10:55:33.494837 Features File:	joint

In [8]:
yap_out_non_o = bclm.read_yap_output(treebank_set=None, tokens_path=bclm.TREEBANK_TOKEN_PATHS['dev'], 
                                     dep_path='data/pruned/dev_pruned_non_o.conll',
                                     map_path='data/pruned/dev_pruned_non_o.map')

In [9]:
yap_out_all = bclm.read_yap_output(treebank_set=None, tokens_path=bclm.TREEBANK_TOKEN_PATHS['dev'], 
                                     dep_path='data/pruned/dev_pruned_all.conll',
                                     map_path='data/pruned/dev_pruned_all.map')

In [10]:
yap_out_non_o_keep = bclm.read_yap_output(treebank_set=None, tokens_path=bclm.TREEBANK_TOKEN_PATHS['dev'], 
                                     dep_path='data/pruned/dev_pruned_non_o_keep.conll',
                                     map_path='data/pruned/dev_pruned_non_o_keep.map')

In [11]:
yap_out_all_keep = bclm.read_yap_output(treebank_set=None, tokens_path=bclm.TREEBANK_TOKEN_PATHS['dev'], 
                                     dep_path='data/pruned/dev_pruned_all_keep.conll',
                                     map_path='data/pruned/dev_pruned_all_keep.map')

In [12]:
dev_gold = bclm.read_dataframe('spmrl', subset='dev')

In [13]:
yap_dev_regular = bclm.read_yap_output()

In [14]:
bclm.evaluate_dfs(dev_gold, yap_dev_regular)

11301 gold tokens/morphems, 11426 predicted, 10445 correct.
Precision: 91.41
Recall:    92.43
F1:        91.92
FP ex.: [(1, 5, 'לישראל', 'NNP'), (1, 8, 'ה', 'DEF'), (2, 11, 'דנה', 'BN'), (3, 4, 'ח"כ', 'NNT'), (3, 28, 'הם', 'S-PRN')]
FN ex.: [(1, 5, 'ישראל', 'NNP'), (1, 5, 'ל', 'PREPOSITION'), (2, 11, 'דנה', 'VB'), (3, 4, 'ח"כ', 'NN'), (3, 28, 'המ', 'S-PRN')]


(91.41431822159986, 92.42544907530306, 91.91710300523606)

In [15]:
bclm.evaluate_dfs(dev_gold, yap_out_non_o)

11301 gold tokens/morphems, 11338 predicted, 10413 correct.
Precision: 91.84
Recall:    92.14
F1:        91.99
FP ex.: [(2, 11, 'דנה', 'BN'), (3, 4, 'ח"כ', 'NNT'), (3, 22, 'מתאילנד', 'NNP'), (3, 28, 'הם', 'S-PRN'), (4, 19, 'פחות', 'NN')]
FN ex.: [(2, 11, 'דנה', 'VB'), (3, 4, 'ח"כ', 'NN'), (3, 22, 'מ', 'PREPOSITION'), (3, 22, 'תאילנד', 'NNP'), (3, 28, 'המ', 'S-PRN')]


(91.84159463750221, 92.14228829307142, 91.99169574627855)

In [16]:
bclm.evaluate_dfs(dev_gold, yap_out_all)

11301 gold tokens/morphems, 10363 predicted, 2388 correct.
Precision: 23.04
Recall:    21.13
F1:        22.05
FP ex.: [(2, 11, 'דנה', 'BN'), (3, 4, 'ח"כ', 'NNT'), (3, 22, 'מתאילנד', 'NNP'), (3, 28, 'הם', 'S-PRN'), (6, 24, 'ה', 'REL')]
FN ex.: [(2, 11, 'דנה', 'VB'), (3, 4, 'ח"כ', 'NN'), (3, 22, 'מ', 'PREPOSITION'), (3, 22, 'תאילנד', 'NNP'), (3, 28, 'המ', 'S-PRN')]


(23.043520216153624, 21.130873374037694, 22.045790251107828)

In [17]:
bclm.evaluate_dfs(dev_gold, yap_out_non_o_keep)

11301 gold tokens/morphems, 11407 predicted, 10475 correct.
Precision: 91.83
Recall:    92.69
F1:        92.26
FP ex.: [(2, 11, 'דנה', 'BN'), (3, 4, 'ח"כ', 'NNT'), (3, 22, 'מתאילנד', 'NNP'), (3, 28, 'הם', 'S-PRN'), (4, 19, 'פחות', 'NN')]
FN ex.: [(2, 11, 'דנה', 'VB'), (3, 4, 'ח"כ', 'NN'), (3, 22, 'מ', 'PREPOSITION'), (3, 22, 'תאילנד', 'NNP'), (3, 28, 'המ', 'S-PRN')]


(91.82957832909617, 92.69091230864525, 92.25823498326581)

In [18]:
bclm.evaluate_dfs(dev_gold, yap_out_all_keep)

11301 gold tokens/morphems, 11263 predicted, 10443 correct.
Precision: 92.72
Recall:    92.41
F1:        92.56
FP ex.: [(2, 11, 'דנה', 'BN'), (3, 4, 'ח"כ', 'NNT'), (3, 22, 'מתאילנד', 'NNP'), (3, 28, 'הם', 'S-PRN'), (6, 24, 'ה', 'REL')]
FN ex.: [(2, 11, 'דנה', 'VB'), (3, 4, 'ח"כ', 'NN'), (3, 22, 'מ', 'PREPOSITION'), (3, 22, 'תאילנד', 'NNP'), (3, 28, 'המ', 'S-PRN')]


(92.71952410547811, 92.40775152641359, 92.5633752880695)

## No FORM

In [19]:
cols = ['sent_id', 'token_id', 'upostag']

In [20]:
bclm.evaluate_dfs(dev_gold, yap_dev_regular, cols=cols)

11301 gold tokens/morphems, 11426 predicted, 10541 correct.
Precision: 92.25
Recall:    93.27
F1:        92.76
FP ex.: [(1, 8, 'DEF'), (2, 11, 'BN'), (3, 4, 'NNT'), (4, 19, 'NN'), (5, 9, 'DEF')]
FN ex.: [(1, 5, 'PREPOSITION'), (2, 11, 'VB'), (3, 4, 'NN'), (4, 19, 'RB'), (5, 9, 'NNT')]


(92.25450726413443, 93.27493142199805, 92.76191314295771)

In [21]:
bclm.evaluate_dfs(dev_gold, yap_out_non_o, cols=cols)

11301 gold tokens/morphems, 11338 predicted, 10505 correct.
Precision: 92.65
Recall:    92.96
F1:        92.8
FP ex.: [(2, 11, 'BN'), (3, 4, 'NNT'), (4, 19, 'NN'), (5, 9, 'DEF'), (5, 9, 'NN')]
FN ex.: [(2, 11, 'VB'), (3, 4, 'NN'), (3, 22, 'PREPOSITION'), (4, 19, 'RB'), (5, 9, 'NNT')]


(92.6530252249074, 92.95637554198744, 92.80445249348469)

In [22]:
bclm.evaluate_dfs(dev_gold, yap_out_all, cols=cols)

11301 gold tokens/morphems, 10363 predicted, 2868 correct.
Precision: 27.68
Recall:    25.38
F1:        26.48
FP ex.: [(2, 11, 'BN'), (3, 4, 'NNT'), (6, 24, 'REL'), (6, 25, 'DEF'), (8, 17, 'DEF')]
FN ex.: [(2, 11, 'VB'), (3, 4, 'NN'), (3, 22, 'PREPOSITION'), (6, 24, 'DEF'), (8, 17, 'VB')]


(27.6753835761845, 25.37828510751261, 26.477104874446084)

In [23]:
bclm.evaluate_dfs(dev_gold, yap_out_non_o_keep, cols=cols)

11301 gold tokens/morphems, 11407 predicted, 10567 correct.
Precision: 92.64
Recall:    93.5
F1:        93.07
FP ex.: [(2, 11, 'BN'), (3, 4, 'NNT'), (4, 19, 'NN'), (5, 9, 'DEF'), (5, 9, 'NN')]
FN ex.: [(2, 11, 'VB'), (3, 4, 'NN'), (3, 22, 'PREPOSITION'), (4, 19, 'RB'), (5, 9, 'NNT')]


(92.63610063995792, 93.50499955756128, 93.06852210674651)

In [24]:
bclm.evaluate_dfs(dev_gold, yap_out_all_keep, cols=cols)

11301 gold tokens/morphems, 11263 predicted, 10538 correct.
Precision: 93.56
Recall:    93.25
F1:        93.41
FP ex.: [(2, 11, 'BN'), (3, 4, 'NNT'), (6, 24, 'REL'), (6, 25, 'DEF'), (8, 17, 'DEF')]
FN ex.: [(2, 11, 'VB'), (3, 4, 'NN'), (3, 22, 'PREPOSITION'), (6, 24, 'DEF'), (8, 17, 'VB')]


(93.5629938737459, 93.24838509866383, 93.40542457011168)

## Evluate Token Means

In [30]:
bclm.evaluate_means(dev_gold, yap_dev_regular)

(11301, 27) (11426, 13)
(8531,) (8531,)


(92.39245106083693, 92.66107920134412, 92.47467220389504)

In [31]:
bclm.evaluate_means(dev_gold, yap_out_non_o)

(11229, 27) (11338, 13)
(8482,) (8482,)


(92.69335062485261, 92.93110901516938, 92.76501498972614)

In [32]:
bclm.evaluate_means(dev_gold, yap_out_all)

(7989, 27) (7995, 13)
(6057,) (6057,)


(29.618623080733038, 29.558087061801775, 29.565319936790967)

In [33]:
bclm.evaluate_means(dev_gold, yap_out_non_o_keep)

(11301, 27) (11407, 13)
(8531,) (8531,)


(92.67377798616809, 92.90724026100888, 92.74349571032258)

In [34]:
bclm.evaluate_means(dev_gold, yap_out_all_keep)

(11301, 27) (11263, 13)
(8531,) (8531,)


(92.95803540030477, 92.89844879459227, 92.89247617931242)

### No FORM

In [25]:
bclm.evaluate_means(dev_gold, yap_dev_regular, cols=cols)

(11301, 27) (11426, 13)
(8531,) (8531,)


/home/nlp/danb/bclm/evaluations.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gold_df['upostag'] = gold_df.upostag.str.replace('_','-')
/home/nlp/danb/bclm/evaluations.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['upostag'] = pred_df.upostag.str.replace('_','-')


(93.07232446372053, 93.42691360919001, 93.13696267394545)

In [26]:
bclm.evaluate_means(dev_gold, yap_out_non_o, cols=cols)

(11229, 27) (11338, 13)
(8482,) (8482,)


(93.32999292619664, 93.67778825748644, 93.39969234569566)

In [27]:
bclm.evaluate_means(dev_gold, yap_out_all, cols=cols)

(7989, 27) (7995, 13)
(6057,) (6057,)


(35.61994386660063, 35.51675747069506, 35.26396062800223)

In [28]:
bclm.evaluate_means(dev_gold, yap_out_non_o_keep, cols=cols)

(11301, 27) (11407, 13)
(8531,) (8531,)


(93.30676356816316, 93.64963075841051, 93.37452763311397)

In [29]:
bclm.evaluate_means(dev_gold, yap_out_all_keep, cols=cols)

(11301, 27) (11263, 13)
(8531,) (8531,)


(93.55976243504082, 93.59785878951276, 93.49967345981882)

## Statistics for reconstructed lattice

In [36]:
gold_tok_train = bclm.get_token_df(bclm.read_dataframe('spmrl', subset='train'), fields=['upostag', 'biose_layer0'])
gold_tok_train.head()

,sent_id,token_id,token_str,upostag,biose_layer0,set
0,501,1,"""",yyQUOT,O,train
1,501,2,תהיה,COP,O,train
2,501,3,נקמה,NN,O,train
3,501,4,ובגדול,CONJ^RB,O^O,train
4,501,5,.,yyDOT,O,train


In [40]:
(gold_tok_train[gold_tok_train.biose_layer0.str.contains('-')]
 .assign(biose_l0 = lambda x: x.biose_layer0.apply(lambda y: '^'.join([z.split('-')[0] for z in y.split('^')])))
 .groupby(['biose_l0', 'upostag']).size()
 .unstack()
 .to_csv('data/train_tok_upostag_biose_layer0.csv')
)